# Function Calling with Microsoft Semantic Kernel in .NET

This notebook demonstrates how to extend the Microsoft Semantic Kernel with custom C# functions and plugins.

Checkout the blog [Extending RAG with database tools](https://binarytrails.com/posts/2025/05/rag-with-tools) for more details.

**Step 1**: Install NuGet packages

To get started with Semantic Kernel, you need to install the required NuGet packages. These packages provide the core functionality for interacting with AI models and managing environment variables. Specifically:
- `Microsoft.SemanticKernel` enables you to build and run AI-powered workflows.
- `DotNetEnv` allows you to load environment variables from a `.env` file, making it easier to manage secrets and configuration settings.

In [21]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.23.0"
#r "nuget: DotNetEnv, 3.1.0"

Installed Packages DotNetEnv, 3.1.0 Microsoft.SemanticKernel, 1.23.0

**Step 2**: Read environment variables

In this step, we load these variables from a `.env` file (if present) so that they can be accessed by the application.


In [22]:
using DotNetEnv;
using System.IO;

var envFilePath = Path.Combine(Environment.CurrentDirectory, "..", ".env");
if (File.Exists(envFilePath))
{
    Env.Load(envFilePath);
    Console.WriteLine($"Loaded environment variables from {envFilePath}");
}
else
{
    Console.WriteLine($"No .env file found at {envFilePath}");
}

Loaded environment variables from d:\personal\ai-examples\01-semantic-kernel-function-calling\..\.env


**Step 3**: Instantiate the Kernel

The Semantic Kernel is the core component that orchestrates AI services and plugins. In this step, we create and configure a Kernel instance, which will be used to interact with AI models.

In [23]:
using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;

//Create Kernel builder
var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = //read from environment variables
    (Environment.GetEnvironmentVariable("USE_AZURE_OPENAI") == "true",
    Environment.GetEnvironmentVariable("MODEL"),
    Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT"),
    Environment.GetEnvironmentVariable("AZURE_OPENAI_API_KEY"),
    Environment.GetEnvironmentVariable("OPENAI_ORG_ID"));

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

**Step 4**: Create Bike Store Plugin

Create a plugin with functions that can be called by the kernel, such as retrieving bike prices or listing bikes by type.

In [24]:
using Microsoft.SemanticKernel;
using System.ComponentModel;
using Kernel = Microsoft.SemanticKernel.Kernel;

public class BikeStoreDatabasePlugin
{ 
    
 [KernelFunction("get_bike_price")]
 [Description("Gets the price of a bike")]
 public int GetBikePrice(string bikeName)
 {
     return bikeName switch
     {
            "Mountain Bike 1" => 500,
            "Mountain Bike 2" => 600,
            "Road Bike 1" => 700,
            "Road Bike 2" => 800,
            "Hybrid Bike 1" => 900,
            "Hybrid Bike 2" => 1000,
         _ => 0
     };
 }

 [KernelFunction("get_bikes_by_type")]
 [Description("Gets the bikes by type")]
 public List<string> GetBikesByType(string bikeType)
 {
     return bikeType switch
     {
         "Mountain" => new List<string> { "Mountain Bike 1", "Mountain Bike 2" },
         "Road" => new List<string> { "Road Bike 1", "Road Bike 2" },
         "Hybrid" => new List<string> { "Hybrid Bike 1", "Hybrid Bike 2" },
         _ => new List<string>()
     };
 }

}

**Step 5**: Register the Plugin and Configure the Kernel

Register the custom plugin with the kernel so its functions are available for function calling.

In [25]:

// Register the plugin
kernel.Plugins.AddFromType<BikeStoreDatabasePlugin>();


**Step 6**: Query the Model for Bike Price

Ask the model for the price of a specific bike. The kernel will use the registered plugin to answer the query.

In [26]:
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

#pragma warning disable SKEXP0001
var chatCompletion = kernel.GetRequiredService<IChatCompletionService>();
OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new() 
{
    FunctionChoiceBehavior = FunctionChoiceBehavior.Auto()
};

#pragma warning restore SKEXP0001

ChatHistory chatHistory = [];
chatHistory.AddUserMessage("What is the price of the bike Mountain Bike 1?");
var response = await chatCompletion.GetChatMessageContentAsync(
    chatHistory,
    executionSettings: openAIPromptExecutionSettings,
    kernel: kernel);

Console.WriteLine(response);


The price of the Mountain Bike 1 is $500.


**Step 7**: Query the Model for Available Bikes by Type

Ask the model for a list of bikes by type. The kernel will use the registered plugin to provide the answer.

In [27]:
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

#pragma warning disable SKEXP0001
var chatCompletion = kernel.GetRequiredService<IChatCompletionService>();
OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new() 
{
    FunctionChoiceBehavior = FunctionChoiceBehavior.Auto()
};

#pragma warning restore SKEXP0001

ChatHistory chatHistory = [];
chatHistory.AddUserMessage("What Mountain bikes do you have?");
var response = await chatCompletion.GetChatMessageContentAsync(
    chatHistory,
    executionSettings: openAIPromptExecutionSettings,
    kernel: kernel);

Console.WriteLine(response);


We have the following mountain bikes available:

1. Mountain Bike 1
2. Mountain Bike 2

Would you like to know more details about any of these bikes?
